Max3-SAT_to_QUBO

Construct QUBO matrix for 3SAT
From:
$$
-g(x,w) = K + [x, w]^T F(phi) [x, w] 
$$

Ex.
$$
-g(x, w) = -4 + 0x_1 - 2x_1x_2 + 2x_1x_3 - x_1w_1 + x_1w_2 - x_1w_3 + x_1w_4 \\
+ x_2 - 0x_2x_3 - x_2w_1 - x_2w_2 + x_2w_3 - x_2w_4 - x_3 - x_3w_1 - x_3w_2 \\
- x_3w_3 + x_3w_4 + 2w_1 + 0(w_1w_2 + w_1w_3 + w_1w_4) \\
+ w_2 + 0(w_2w_3 + w_2w_4) + w_3 + 0w_3w_4 + 0w_4\\
$$\\


Ex. $$φ = (x1 ∨ x2 ∨ x3) ∧ (-x1 ∨ x2 ∨ x3) ∧ (x1 ∨ -x2 ∨ x3) ∧ (-x1 ∨ x2 ∨ -x3)$$

In [1]:
import numpy as np
from numpy.typing import NDArray
import sympy
import re

In [2]:
num_x = 3
num_m = 4
phi_matrix = np.zeros((num_m, num_x), dtype=int)
display(phi_matrix) # phi_matrix[clause][x]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

x presents in each clause

In [3]:
phi_matrix[0] = 1
phi_matrix[1] = 1
phi_matrix[2] = 1
phi_matrix[3] = 1
display(phi_matrix)

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

not x in each clause

In [4]:
phi_matrix[1][0] = -1
phi_matrix[2][1] = -1
phi_matrix[3][0] = -1
phi_matrix[3][2] = -1
display(phi_matrix)

array([[ 1,  1,  1],
       [-1,  1,  1],
       [ 1, -1,  1],
       [-1,  1, -1]])

In [5]:
# # phi = (x0 or x1 or x2) and (x0 or -x1 or x2)
# num_x = 3
# num_m = 2
# phi_matrix = np.zeros((num_m, num_x), dtype=int)
# phi_matrix[:] = 1
# phi_matrix[1][1] = -1
# display(phi_matrix) # phi_matrix[clause][x]

In [6]:
clauses = np.array([[1, 2, 3],
                    [-1, 2, 3],
                    [1, -2, 3],
                    [-1, 2, -3]])

In [7]:
# def problem_3SAT_to_QUBO(phi_matrix: NDArray[np.int_]) -> NDArray[np.int_]: # phi_matrix[clause][x]
#     num_x = phi_matrix.shape[1] # Phi matrix Column
#     x = sympy.symbols(f'x0:{num_x}') # x0, x1, ... x_num_x -1
#     num_m = phi_matrix.shape[0] # Phi matrix Row
#     w = sympy.symbols(f'w0:{num_m}') # w0, w1, ... w_num_m - 1
#     QUBO_matrix = np.zeros((num_x + num_m, num_x + num_m), dtype=int)
#     sum_g = 0

#     error = False
    
#     for i in range(num_m):
#         x_array = phi_matrix[i]
#         x_presents_indices = np.where(x_array != 0)[0]
#         # Make sure each clause has only 3 literals
#         if (x_presents_indices.shape[0] != 3):
#             error = True
#             break

#         y_i1 = x[x_presents_indices[0]] if (x_array[x_presents_indices[0]] == 1) else (1 + (-1 * x[x_presents_indices[0]]))
#         y_i2 = x[x_presents_indices[1]] if (x_array[x_presents_indices[1]] == 1) else (1 + (-1 * x[x_presents_indices[1]]))
#         y_i3 = x[x_presents_indices[2]] if (x_array[x_presents_indices[2]] == 1) else (1 + (-1 * x[x_presents_indices[2]]))
#         sum_g += y_i1 + y_i2 + y_i3 + (w[i] * y_i1) + (w[i] * y_i2) + (w[i] * y_i3) - (y_i1 * y_i2) - (y_i1 * y_i3) - (y_i2 * y_i3) - 2 * w[i]
    
#     sum_neg_g = sympy.simplify(-1 * sum_g)
#     sum_neg_g_dict = {str(term): coefficient for term, coefficient in sum_neg_g.as_coefficients_dict().items()}

#     # for i in range(num_x + num_m):
#     #     for j in range(i, num_x + num_m):
#     #             if (i == j and i >= num_x): # Skip w[i] * w[j] (These terms do not appear), Only consider w[i==j]
#     #                 QUBO_matrix[i][j] = 1
#     #                 break
#     #             elif (i == j): # x[i]
#     #                 QUBO_matrix[i][j] = 2
#     #             elif (j < num_x): # x[i] * x[j]
#     #                 QUBO_matrix[i][j] = 3
#     #             elif (j >= num_x): # x[i] * w[j]
#     #                 QUBO_matrix[i][j] = 4

#     for term in sum_neg_g_dict:
#         # print(f'{term}: {sum_neg_g_dict[term]}')
#         if re.match(r'^w\d+$', term): # w[i]
#             i = int(term[1:]) + num_x
#             j = int(term[1:]) + num_x
#             QUBO_matrix[i][j] = sum_neg_g_dict[term]
#         elif re.match(r'^x\d+$', term): # x[i]
#             i = int(term[1:])
#             j = int(term[1:])
#             QUBO_matrix[i][j] = sum_neg_g_dict[term]
#         elif re.match(r'^w(0|[1-9][0-9]*)\*x(\d+)$', term): # w[i] * x[j]
#             match = re.match(r'^w(0|[1-9][0-9]*)\*x(\d+)$', term)
#             w_number = int(match.group(1))
#             x_number = int(match.group(2))
#             QUBO_matrix[x_number][w_number + num_x] = sum_neg_g_dict[term]
#         elif re.match(r'^x(0|[1-9][0-9]*)\*x(\d+)$', term): # x[i] * x[j]
#             match = re.match(r'^x(0|[1-9][0-9]*)\*x(\d+)$', term)
#             first_x_number = int(match.group(1))
#             second_x_number = int(match.group(2))
#             QUBO_matrix[first_x_number][second_x_number] = sum_neg_g_dict[term]
    
#     if (error):
#         return "An error has occured, Make sure phi_matrix is of 3SAT kind (Only 3 literals in each clauses)"
#     else:
#         return QUBO_matrix

In [8]:
# QUBO_matrix = problem_3SAT_to_QUBO(phi_matrix)
# print(QUBO_matrix)

In [9]:
def problem_3SAT_to_QUBO(clauses: NDArray[np.int_], num_literals: int) -> tuple[NDArray[np.int_], int]:
    num_x = num_literals 
    x = sympy.symbols(f'x0:{num_x}') # x0, x1, ... x_num_x -1
    num_m = clauses.shape[0] # clauses matrix Row
    w = sympy.symbols(f'w0:{num_m}') # w0, w1, ... w_num_m - 1
    QUBO_matrix = np.zeros((num_x + num_m, num_x + num_m), dtype=int)
    sum_g = 0

    error = False
    
    for i in range(num_m):
        x_array = clauses[i]
        # Make sure each clause is within specified literals.
        if (x_array[np.abs(x_array) > num_x].size > 0):
            error = True
            break

        y_i1 = x[x_array[0] - 1] if (x_array[0] > 0) else (1 + (-1 * x[x_array[0] * -1 - 1]))
        y_i2 = x[x_array[1] - 1] if (x_array[1] > 0) else (1 + (-1 * x[x_array[1] * -1 - 1]))
        y_i3 = x[x_array[2] - 1] if (x_array[2] > 0) else (1 + (-1 * x[x_array[2] * -1 - 1]))
        sum_g += y_i1 + y_i2 + y_i3 + (w[i] * y_i1) + (w[i] * y_i2) + (w[i] * y_i3) - (y_i1 * y_i2) - (y_i1 * y_i3) - (y_i2 * y_i3) - 2 * w[i]
    
    sum_neg_g = sympy.simplify(-1 * sum_g)
    sum_neg_g_dict = {str(term): coefficient for term, coefficient in sum_neg_g.as_coefficients_dict().items()}

    # for i in range(num_x + num_m):
    #     for j in range(i, num_x + num_m):
    #             if (i == j and i >= num_x): # Skip w[i] * w[j] (These terms do not appear), Only consider w[i==j]
    #                 QUBO_matrix[i][j] = 1
    #                 break
    #             elif (i == j): # x[i]
    #                 QUBO_matrix[i][j] = 2
    #             elif (j < num_x): # x[i] * x[j]
    #                 QUBO_matrix[i][j] = 3
    #             elif (j >= num_x): # x[i] * w[j]
    #                 QUBO_matrix[i][j] = 4

    for term in sum_neg_g_dict:
        # print(f'{term}: {sum_neg_g_dict[term]}')
        if re.match(r'^w\d+$', term): # w[i]
            i = int(term[1:]) + num_x
            j = int(term[1:]) + num_x
            QUBO_matrix[i][j] = sum_neg_g_dict[term]
        elif re.match(r'^x\d+$', term): # x[i]
            i = int(term[1:])
            j = int(term[1:])
            QUBO_matrix[i][j] = sum_neg_g_dict[term]
        elif re.match(r'^w(0|[1-9][0-9]*)\*x(\d+)$', term): # w[i] * x[j]
            match = re.match(r'^w(0|[1-9][0-9]*)\*x(\d+)$', term)
            w_number = int(match.group(1))
            x_number = int(match.group(2))
            QUBO_matrix[x_number][w_number + num_x] = sum_neg_g_dict[term]
        elif re.match(r'^x(0|[1-9][0-9]*)\*x(\d+)$', term): # x[i] * x[j]
            match = re.match(r'^x(0|[1-9][0-9]*)\*x(\d+)$', term)
            first_x_number = int(match.group(1))
            second_x_number = int(match.group(2))
            QUBO_matrix[first_x_number][second_x_number] = sum_neg_g_dict[term]
        elif re.match(r'^1$', term):
            K = sum_neg_g_dict[term]
    
    if (error):
        return "An error has occured, Make sure the literals in each clause are within the specified literals."
    else:
        return QUBO_matrix, K

In [10]:
QUBO_matrix, K = problem_3SAT_to_QUBO(clauses, 3)
print(QUBO_matrix)
print("K = ", K)

[[ 0 -2  2 -1  1 -1  1]
 [ 0  1  0 -1 -1  1 -1]
 [ 0  0 -1 -1 -1 -1  1]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0]]
K =  -3


In [11]:
def bruteForce(QUBO_matrix: NDArray[np.int_]) -> tuple[NDArray[np.int_], int]:
    num_x = QUBO_matrix.shape[0]
    min_energy = np.inf
    min_x = np.zeros(num_x, dtype=int)
    for i in range(2 ** num_x):
        x = np.array([int(x) for x in bin(i)[2:].zfill(num_x)]) # Convert to binary and fill with 0s
        energy = np.sum(np.dot(x, QUBO_matrix).dot(x))
        if (energy < min_energy):
            min_energy = energy
            min_x = x
    return min_x, min_energy

In [12]:
min_x, min_energy = bruteForce(QUBO_matrix)
print(min_x)
print(min_energy)
print("Max clauses satisfied: ", (K + min_energy) * -1)

[0 0 1 0 0 0 0]
-1
Max clauses satisfied:  4


Solve for QUBO(F(phi)) -> min [x, w]^T F(phi) [x, w]

In [13]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization import QuadraticProgram

qp = QuadraticProgram()
for i in range(QUBO_matrix.shape[0]):
    qp.binary_var(f'x{i}')


qp.minimize(quadratic=QUBO_matrix)
print(qp.prettyprint())
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()


exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

result = exact.solve(qubo)
print(result.prettyprint())
print(result.variables_dict)

Problem name: 

Minimize
  -2*x0*x1 + 2*x0*x2 - x0*x3 + x0*x4 - x0*x5 + x0*x6 + x1^2 - x1*x3 - x1*x4
  + x1*x5 - x1*x6 - x2^2 - x2*x3 - x2*x4 - x2*x5 + x2*x6 + 2*x3^2 + x4^2 + x5^2

Subject to
  No constraints

  Binary variables (7)
    x0 x1 x2 x3 x4 x5 x6

objective function value: -1.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=1.0, x6=0.0
status: SUCCESS
{'x0': 1.0, 'x1': 1.0, 'x2': 1.0, 'x3': 1.0, 'x4': 1.0, 'x5': 1.0, 'x6': 0.0}


In [14]:
from amplify import VariableGenerator, solve
import sys
sys.path.append('../') # To use Utils
from Utils.solvers import GetFixstarClient, GetGurobiClient, GetDWaveClient

QUBO_matrix to model

In [15]:
def QUBO_matrix_to_model(QUBO_matrix: np.ndarray, weight=1):
    size = QUBO_matrix.shape[0] # [n+m]
    gen = VariableGenerator()
    model = gen.matrix("Binary", size)
    q = model.quadratic
    for i in range(size):
        for j in range(size):
            q[i][j] = QUBO_matrix[i][j]
    return model

In [16]:
model = QUBO_matrix_to_model(QUBO_matrix)
print(model)

(x^T) Q x + (p^T) x + c
where:
  x = [q_0, q_1, q_2, q_3, q_4, q_5, q_6],
  Q = [[ 0., -2.,  2., -1.,  1., -1.,  1.],
       [ 0.,  1.,  0., -1., -1.,  1., -1.],
       [ 0.,  0., -1., -1., -1., -1.,  1.],
       [ 0.,  0.,  0.,  2.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]],
  p = [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
  c = 0


FixStar

In [17]:
clientFS = GetFixstarClient()
resultFS = solve(model, clientFS)

print(resultFS.best.objective)
print(resultFS.best.values)
print("Max clauses satisfied: ", ((K + resultFS.best.objective) * -1))

-1.0
{q_0: 1, q_1: 1, q_2: 1, q_3: 1, q_4: 1, q_5: 0, q_6: 0}
Max clauses satisfied:  4.00000000000000


Gurobi

In [18]:
clientG = GetGurobiClient()
resultG = solve(model, clientG)

print(resultG.best.objective)
print(resultG.best.values)
print("Max clauses satisfied: ", ((K + resultG.best.objective) * -1))

-1.0
{q_0: 1, q_1: 1, q_2: 0, q_3: 1, q_4: 0, q_5: 0, q_6: 0}
Max clauses satisfied:  4.00000000000000


In [19]:
clientDWave = GetDWaveClient()
resultDWave = solve(model, clientDWave)

print(resultDWave.best.objective)
print(resultDWave.best.values)
print("Max clauses satisfied: ", ((K + resultDWave.best.objective) * -1))

-1.0
{q_0: 1, q_1: 1, q_2: 1, q_3: 1, q_4: 1, q_5: 1, q_6: 0}
Max clauses satisfied:  4.00000000000000


In [20]:
from pyqubo import Array, solve_qubo
#-2*x0*x1 + 2*x0*x2 - x0*x3 + x0*x4 - x0*x5 + x0*x6 + x1^2 - x1*x3 - x1*x4 + x1*x5 - x1*x6 - x2^2 - x2*x3 - x2*x4 - x2*x5 + x2*x6 + 2*x3^2 + x4^2 + x5^2
# Define binary variables
x = Array.create('x', shape=7, vartype='BINARY')

# Create a QUBO model
H = 2*x[0]*x[1] + 2*x[0]*x[2] - x[0]*x[3] + x[0]*x[4] - x[0]*x[5] + x[0]*x[6] + x[1]**2 - x[1]*x[3] - x[1]*x[4] + x[1]*x[5] - x[1]*x[6] - x[2]**2 - x[2]*x[3] - x[2]*x[4] - x[2]*x[5] + x[2]*x[6] + 2*x[3]**2 + x[4]**2 + x[5]**2

# Compile the model
model = H.compile()
qubo, offset = model.to_qubo()
print("QUBO matrix:", qubo)

solution = solve_qubo(qubo)
print("Solution:", solution)

energy = model.energy(solution, vartype='BINARY')

# Print the minimum energy (value of the equation)
print("Minimum value of the equation (energy):", energy)
print("Max clauses satisfied: ", ((K + energy) * -1))

QUBO matrix: {('x[5]', 'x[1]'): 1.0, ('x[5]', 'x[2]'): -1.0, ('x[2]', 'x[6]'): 1.0, ('x[5]', 'x[0]'): -1.0, ('x[4]', 'x[2]'): -1.0, ('x[2]', 'x[2]'): -1.0, ('x[4]', 'x[1]'): -1.0, ('x[5]', 'x[5]'): 1.0, ('x[6]', 'x[0]'): 1.0, ('x[6]', 'x[1]'): -1.0, ('x[3]', 'x[1]'): -1.0, ('x[4]', 'x[4]'): 1.0, ('x[3]', 'x[3]'): 2.0, ('x[1]', 'x[0]'): 2.0, ('x[2]', 'x[0]'): 2.0, ('x[3]', 'x[2]'): -1.0, ('x[4]', 'x[0]'): 1.0, ('x[1]', 'x[1]'): 1.0, ('x[3]', 'x[0]'): -1.0}
Solution: {'x[0]': 0, 'x[1]': 1, 'x[2]': 1, 'x[3]': 0, 'x[4]': 1, 'x[5]': 0, 'x[6]': 1}
Minimum value of the equation (energy): -1.0
Max clauses satisfied:  4.00000000000000


C:\Users\Asus\AppData\Local\Temp\ipykernel_20212\3341007119.py:14: DeprecationWarning: Call to deprecated function (or staticmethod) solve_qubo. (You should use simulated annealing sampler of dwave-neal directly.) -- Deprecated since version 0.4.0.
  solution = solve_qubo(qubo)


CNF to clauses

In [21]:
def read_cnf_file(filename: str) -> tuple[NDArray[np.int_], int, int]:
    clauses = []

    with open(filename, 'r') as file:
        for line in file:
            # Skip comment lines
            if line.startswith('c'):
                continue
            
            # Read the problem line to get the number of variables
            if line.startswith('p cnf'):
                parts = line.split()
                num_n = int(parts[2])
                num_m = int(parts[3])
                continue
            
            # Read clauses
            clause = [x for x in line.split() if x != "0"]  # Skip the trailing 0
            if (len(clause) == 3):
                clauses.append(clause)
    
    return np.array(clauses, dtype="int"), num_n, num_m


In [22]:
# def clauses_to_phi_matrix(clauses: NDArray[np.int_], num_n: int) -> NDArray[np.int_]:
#     phi_matrix = np.zeros((clauses.shape[0], num_n), dtype=int)

#     # Populate the matrix based on literals in each clause
#     for i, clause in enumerate(clauses):
#         for literal in clause:
#             if literal > 0:
#                 phi_matrix[i, literal - 1] = 1  # Positive literal
#             elif literal < 0:
#                 phi_matrix[i, abs(literal) - 1] = -1  # Negative literal
                
#     return np.array(phi_matrix, dtype="int")

In [23]:
filename = 'TestSets\\Satisfiable\\uf20-91\\uf20-01.cnf'
clauses, num_n, num_m = read_cnf_file(filename)

In [24]:
print(num_n, num_m)

20 91


In [25]:
print("Clauses:")
print(clauses)

Clauses:
[[  4 -18  19]
 [  3  18  -5]
 [ -5  -8 -15]
 [-20   7 -16]
 [ 10 -13  -7]
 [-12  -9  17]
 [ 17  19   5]
 [-16   9  15]
 [ 11  -5 -14]
 [ 18 -10  13]
 [ -3  11  12]
 [ -6 -17  -8]
 [-18  14   1]
 [-19 -15  10]
 [ 12  18 -19]
 [ -8   4   7]
 [ -8  -9   4]
 [  7  17 -15]
 [ 12  -7 -14]
 [-10 -11   8]
 [  2 -15 -11]
 [  9   6   1]
 [-11  20 -17]
 [  9 -15  13]
 [ 12  -7 -17]
 [-18  -2  20]
 [ 20  12   4]
 [ 19  11  14]
 [-16  18  -4]
 [ -1 -17 -19]
 [-13  15  10]
 [-12 -14 -13]
 [ 12 -14  -7]
 [ -7  16  10]
 [  6  10   7]
 [ 20  14 -16]
 [-19  17  11]
 [ -7   1 -20]
 [ -5  12  15]
 [ -4  -9 -13]
 [ 12 -11  -7]
 [ -5  19  -8]
 [  1  16  17]
 [ 20 -14 -15]
 [ 13  -4  10]
 [ 14   7  10]
 [ -5   9  20]
 [ 10   1 -19]
 [-16 -15  -1]
 [ 16   3 -11]
 [-15 -10   4]
 [  4 -15  -3]
 [-10 -16  11]
 [ -8  12  -5]
 [ 14  -6  12]
 [  1   6  11]
 [-13  -5  -1]
 [ -7  -2  12]
 [  1 -20  19]
 [ -2 -13  -8]
 [ 15  18   4]
 [-11  14   9]
 [ -6 -15  -2]
 [  5 -12 -15]
 [ -6  17   5]
 [-13   5 -19]
 

In [26]:
# print("\nMatrix:")
# print(phi_matrix)

In [27]:
print(clauses.shape)
print(len(np.unique(clauses)))

(91, 3)
40


In [28]:
QUBO_matrix, K = problem_3SAT_to_QUBO(clauses, 20)
print(QUBO_matrix)
print("K = ", K)

[[-3  0  0 ...  0  0  0]
 [ 0  2 -1 ...  0  1  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  0]]
K =  -70


In [29]:
print(QUBO_matrix.shape)

(111, 111)


Qiskit, too many var

In [30]:
# qp = QuadraticProgram()
# for i in range(QUBO_matrix.shape[0]):
#     qp.binary_var(f'x{i}')

# qp.minimize(quadratic=QUBO_matrix)
# print(qp.prettyprint())
# qp2qubo = QuadraticProgramToQubo()
# qubo = qp2qubo.convert(qp)
# qubitOp, offset = qubo.to_ising()


# exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

# result = exact.solve(qubo)
# print(result.prettyprint())
# print(result.variables_dict)

In [31]:
model_uf20_01 = QUBO_matrix_to_model(QUBO_matrix)
print(model_uf20_01)

(x^T) Q x + (p^T) x + c
where:
  x = [    q_0,     q_1,     q_2,     q_3,     q_4,     q_5,     q_6,     q_7, 
           q_8,     q_9,  q_{10},  q_{11},  q_{12},  q_{13},  q_{14},  q_{15}, 
        q_{16},  q_{17},  q_{18},  q_{19},  q_{20},  q_{21},  q_{22},  q_{23}, 
        q_{24},  q_{25},  q_{26},  q_{27},  q_{28},  q_{29},  q_{30},  q_{31}, 
        q_{32},  q_{33},  q_{34},  q_{35},  q_{36},  q_{37},  q_{38},  q_{39}, 
        q_{40},  q_{41},  q_{42},  q_{43},  q_{44},  q_{45},  q_{46},  q_{47}, 
        q_{48},  q_{49},  q_{50},  q_{51},  q_{52},  q_{53},  q_{54},  q_{55}, 
        q_{56},  q_{57},  q_{58},  q_{59},  q_{60},  q_{61},  q_{62},  q_{63}, 
        q_{64},  q_{65},  q_{66},  q_{67},  q_{68},  q_{69},  q_{70},  q_{71}, 
        q_{72},  q_{73},  q_{74},  q_{75},  q_{76},  q_{77},  q_{78},  q_{79}, 
        q_{80},  q_{81},  q_{82},  q_{83},  q_{84},  q_{85},  q_{86},  q_{87}, 
        q_{88},  q_{89},  q_{90},  q_{91},  q_{92},  q_{93},  q_{94},  q_{95}, 
        q

In [32]:
clientFS = GetFixstarClient()
resultFS = solve(model_uf20_01, clientFS)

min_energy = resultFS.best.objective
print(min_energy)
print(resultFS.best.values)
print("Max clauses satisfied: ", ((K + min_energy) * -1))

2024-12-13 16:51:27 [amplify:WARNING] Connection timeout, retrying after sleep 1 second (1/3)
2024-12-13 16:51:31 [amplify:WARNING] SSL connection failed, retrying after sleep 1 second (1/3)


-21.0
{q_0: 1, q_1: 0, q_2: 0, q_3: 1, q_4: 0, q_5: 0, q_6: 0, q_7: 0, q_8: 0, q_9: 1, q_{10}: 0, q_{11}: 0, q_{12}: 1, q_{13}: 1, q_{14}: 1, q_{15}: 0, q_{16}: 1, q_{17}: 0, q_{18}: 0, q_{19}: 1, q_{20}: 0, q_{21}: 0, q_{22}: 0, q_{23}: 0, q_{24}: 1, q_{25}: 1, q_{26}: 0, q_{27}: 0, q_{28}: 0, q_{29}: 0, q_{30}: 0, q_{31}: 1, q_{32}: 1, q_{33}: 1, q_{34}: 0, q_{35}: 0, q_{36}: 1, q_{37}: 0, q_{38}: 0, q_{39}: 0, q_{40}: 0, q_{41}: 0, q_{42}: 1, q_{43}: 0, q_{44}: 0, q_{45}: 1, q_{46}: 0, q_{47}: 0, q_{48}: 0, q_{49}: 0, q_{50}: 0, q_{51}: 0, q_{52}: 0, q_{53}: 0, q_{54}: 0, q_{55}: 1, q_{56}: 0, q_{57}: 0, q_{58}: 1, q_{59}: 0, q_{60}: 0, q_{61}: 0, q_{62}: 1, q_{63}: 0, q_{64}: 0, q_{65}: 1, q_{66}: 1, q_{67}: 1, q_{68}: 0, q_{69}: 0, q_{70}: 0, q_{71}: 1, q_{72}: 0, q_{73}: 0, q_{74}: 0, q_{75}: 0, q_{76}: 0, q_{77}: 1, q_{78}: 0, q_{79}: 0, q_{80}: 1, q_{81}: 1, q_{82}: 1, q_{83}: 0, q_{84}: 1, q_{85}: 0, q_{86}: 0, q_{87}: 0, q_{88}: 0, q_{89}: 0, q_{90}: 0, q_{91}: 1, q_{92}: 0, 

In [33]:
clientG = GetGurobiClient()
resultG = solve(model_uf20_01, clientG)

min_energy = resultG.best.objective
print(min_energy)
print(resultG.best.values)
print("Max clauses satisfied: ", ((K + min_energy) * -1))

-21.0
{q_0: 1, q_1: 0, q_2: 0, q_3: 1, q_4: 0, q_5: 1, q_6: 0, q_7: 0, q_8: 0, q_9: 0, q_{10}: 0, q_{11}: 0, q_{12}: 1, q_{13}: 1, q_{14}: 1, q_{15}: 0, q_{16}: 1, q_{17}: 0, q_{18}: 0, q_{19}: 1, q_{20}: 0, q_{21}: 0, q_{22}: 0, q_{23}: 0, q_{24}: 0, q_{25}: 1, q_{26}: 0, q_{27}: 1, q_{28}: 0, q_{29}: 1, q_{30}: 0, q_{31}: 0, q_{32}: 1, q_{33}: 0, q_{34}: 0, q_{35}: 0, q_{36}: 1, q_{37}: 0, q_{38}: 0, q_{39}: 0, q_{40}: 0, q_{41}: 1, q_{42}: 0, q_{43}: 0, q_{44}: 0, q_{45}: 1, q_{46}: 0, q_{47}: 0, q_{48}: 0, q_{49}: 0, q_{50}: 0, q_{51}: 0, q_{52}: 0, q_{53}: 0, q_{54}: 0, q_{55}: 1, q_{56}: 0, q_{57}: 0, q_{58}: 1, q_{59}: 0, q_{60}: 0, q_{61}: 0, q_{62}: 1, q_{63}: 0, q_{64}: 0, q_{65}: 0, q_{66}: 1, q_{67}: 1, q_{68}: 0, q_{69}: 0, q_{70}: 0, q_{71}: 0, q_{72}: 0, q_{73}: 0, q_{74}: 0, q_{75}: 0, q_{76}: 0, q_{77}: 0, q_{78}: 0, q_{79}: 0, q_{80}: 0, q_{81}: 0, q_{82}: 0, q_{83}: 0, q_{84}: 0, q_{85}: 0, q_{86}: 0, q_{87}: 0, q_{88}: 0, q_{89}: 0, q_{90}: 0, q_{91}: 1, q_{92}: 0, 

In [34]:
clientDWave = GetDWaveClient()
resultDWave = solve(model_uf20_01, clientDWave)

min_energy = resultDWave.best.objective
print(min_energy)
print(resultDWave.best.values)
print("Max clauses satisfied: ", ((K + min_energy) * -1))

-16.0
{q_0: 1, q_1: 0, q_2: 1, q_3: 1, q_4: 0, q_5: 0, q_6: 0, q_7: 1, q_8: 1, q_9: 1, q_{10}: 1, q_{11}: 1, q_{12}: 0, q_{13}: 1, q_{14}: 0, q_{15}: 0, q_{16}: 1, q_{17}: 1, q_{18}: 0, q_{19}: 1, q_{20}: 0, q_{21}: 1, q_{22}: 1, q_{23}: 0, q_{24}: 1, q_{25}: 0, q_{26}: 0, q_{27}: 0, q_{28}: 0, q_{29}: 0, q_{30}: 1, q_{31}: 0, q_{32}: 1, q_{33}: 1, q_{34}: 0, q_{35}: 0, q_{36}: 0, q_{37}: 1, q_{38}: 0, q_{39}: 0, q_{40}: 0, q_{41}: 1, q_{42}: 1, q_{43}: 0, q_{44}: 0, q_{45}: 1, q_{46}: 1, q_{47}: 1, q_{48}: 1, q_{49}: 0, q_{50}: 0, q_{51}: 0, q_{52}: 0, q_{53}: 0, q_{54}: 0, q_{55}: 0, q_{56}: 1, q_{57}: 1, q_{58}: 1, q_{59}: 0, q_{60}: 1, q_{61}: 0, q_{62}: 0, q_{63}: 0, q_{64}: 0, q_{65}: 1, q_{66}: 1, q_{67}: 1, q_{68}: 0, q_{69}: 0, q_{70}: 1, q_{71}: 0, q_{72}: 1, q_{73}: 0, q_{74}: 1, q_{75}: 0, q_{76}: 0, q_{77}: 1, q_{78}: 0, q_{79}: 0, q_{80}: 1, q_{81}: 0, q_{82}: 1, q_{83}: 0, q_{84}: 1, q_{85}: 0, q_{86}: 1, q_{87}: 0, q_{88}: 0, q_{89}: 0, q_{90}: 0, q_{91}: 0, q_{92}: 1, 

In [35]:
result_array_Dwave = np.array(list(resultDWave.best.values.values()))
print(result_array_Dwave)

[1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.]


In [36]:
def verifier(clauses: NDArray[np.int_], result_values: NDArray[np.int_]) -> tuple[bool, int, list]:
    num_clauses_True = 0
    wrong_clause = []
    for clause in clauses:
        clause_Truth_value = False
        for literal in clause:
            if literal > 0 and result_values[literal - 1] == 1:
                clause_Truth_value = True
                break
            elif literal < 0 and result_values[abs(literal) - 1] == 0:
                clause_Truth_value = True
                break
        if clause_Truth_value:
            num_clauses_True += 1
        else:
            wrong_clause.append(clause)
    return (num_clauses_True == clauses.shape[0]), num_clauses_True, wrong_clause

In [37]:
print(verifier(clauses, result_array_Dwave))

(False, 89, [array([15, -9, 13]), array([  2, -10, -18])])


Test Benchmarks

In [38]:
problem_num = "075"
filename = f'TestSets\\Satisfiable\\uf100-430\\uf100-{problem_num}.cnf'
clauses, num_n, num_m = read_cnf_file(filename)
print("Number of literals:", num_n)
print("Number of clauses:", num_m)


Number of literals: 100
Number of clauses: 430


In [39]:
QUBO_matrix, K = problem_3SAT_to_QUBO(clauses, num_n)
print(QUBO_matrix)
print("K = ", K)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
K =  -330


In [40]:
print("QUBO shape:", QUBO_matrix.shape)
print("QUBO size:", QUBO_matrix.size)

QUBO shape: (530, 530)
QUBO size: 280900


In [41]:
print(f"Number of 0:", np.sum(QUBO_matrix == 0))
print(f"Number of non-0:", np.sum(QUBO_matrix != 0))
print(f"0 to non-0 %", np.sum(QUBO_matrix == 0) / (np.sum(QUBO_matrix != 0) + np.sum(QUBO_matrix == 0)) * 100)

model = QUBO_matrix_to_model(QUBO_matrix)

Number of 0: 278169
Number of non-0: 2731
0 to non-0 % 99.02776788892844


In [42]:
clientFS = GetFixstarClient()
resultFS = solve(model, clientFS)
literal_resultFS = np.array(list(resultFS.best.values.values()))

print(f"FixStar result for P-{problem_num}:", verifier(clauses, literal_resultFS))
print("FixStar obj value:", resultFS.best.objective)
print("Max clauses satisfied: ", ((K + resultFS.best.objective) * -1))

FixStar result for P-075: (True, 430, [])
FixStar obj value: -100.0
Max clauses satisfied:  430.000000000000


In [43]:
clientG = GetGurobiClient()
resultG = solve(model, clientG)
literal_resultG = np.array(list(resultG.best.values.values()))

print(f"Gurobi result for P-{problem_num}:", verifier(clauses, literal_resultG))
print("Gurobi obj value:", resultG.best.objective)
print("Max clauses satisfied: ", ((K + resultG.best.objective) * -1))

Gurobi result for P-075: (False, 428, [array([-86, -19,  56]), array([-69,  31,  66])])
Gurobi obj value: -98.0
Max clauses satisfied:  428.000000000000


In [44]:
clientDWave = GetDWaveClient()
resultDWave = solve(model, clientDWave)
literal_resultDWave = np.array(list(resultDWave.best.values.values()))

print(f"DWave result for P-{problem_num}:", verifier(clauses, literal_resultDWave))
print("DWave obj value:", resultDWave.best.objective)
print("Max clauses satisfied: ", ((K + resultDWave.best.objective) * -1))

DWave result for P-075: (False, 414, [array([ 93,  15, -99]), array([ 63,  39, -71]), array([ 76,  13, -37]), array([13, -8, 62]), array([-18,  82,  -4]), array([-89,  56, -60]), array([-17,  44,  63]), array([ -8, -66, -69]), array([-66, -92,  40]), array([46, 23,  5]), array([ 68,  30, -41]), array([-79,  27,  43]), array([ 14,  13, -89]), array([-98,  58,  -4]), array([63, 81, -4]), array([ 35,   9, -95])])
DWave obj value: -71.0
Max clauses satisfied:  401.000000000000
